In [ ]:
# ── Jupyter settings ──────────────────────────────────────────────────────────
%load_ext autoreload
%autoreload 2

In [ ]:
import asyncio
from pylabrobot.liquid_handling import LiquidHandler
from pylabrobot.liquid_handling.backends import STARBackend
from pylabrobot.resources.hamilton import (
    STARLetDeck, MFX_CAR_L5_base, TIP_CAR_480_A00
)
from pylabrobot.resources.hamilton.mfx_modules import MFX_DWP_module_188042
from pylabrobot.resources import (
    HTF, TIP_50ul_w_filter
       # target qPCR plate (unchanged)
)

In [ ]:
from typing import Optional

from pylabrobot.resources.height_volume_functions import (
  compute_height_from_volume_rectangle,
  compute_volume_from_height_rectangle,
)
from pylabrobot.resources.plate import Lid, Plate
from pylabrobot.resources.utils import create_ordered_items_2d
from pylabrobot.resources.well import (
  CrossSectionType,
  Well,
  WellBottomType,
)

def BioER_96_wellplate_Vb_2200ul(name: str, lid: Optional[Lid] = None) -> Plate:
  """
  BioER plate similar to NEST and other KingFisher-compatible deep well plates
  https://en.bioer.com/uploadfiles/2024/05/20240513165756879.pdf
  Cat no. BSH06M1T-A
  """
  INNER_WELL_WIDTH = 8.2  # from spec  
  INNER_WELL_LENGTH = 8.2  # from spec

  well_kwargs = {
    "size_x": INNER_WELL_WIDTH,  # measured
    "size_y": INNER_WELL_LENGTH,  # measured
    "size_z": 42.4,  # from spec
    "bottom_type": WellBottomType.V,
    "cross_section_type": CrossSectionType.RECTANGLE,
    "compute_height_from_volume": lambda liquid_volume: compute_height_from_volume_rectangle(
      liquid_volume,
      INNER_WELL_LENGTH,
      INNER_WELL_WIDTH,
    ),
    "compute_volume_from_height": lambda liquid_height: compute_volume_from_height_rectangle(
      liquid_height,
      INNER_WELL_LENGTH,
      INNER_WELL_WIDTH,
    ),
    # "material_z_thickness": 1,
    "material_z_thickness": 0.80, # measured
  }

  return Plate(
    name=name,
    size_x=127.30,  # from spec
    size_y=85.20,  # from spec
    size_z=44.2,  # from spec
    lid=lid,
    model=BioER_96_wellplate_Vb_2200ul.__name__,
    ordered_items=create_ordered_items_2d(
      Well,
      num_items_x=12,
      num_items_y=8,
      dx=9.2,  # measured
      dy=9.5,  # measured
      dz=1.2,  # from spec
      item_dx=9, # from spec
      item_dy=9, # from spec
      **well_kwargs,
    ),
  )

def VWR_96_wellplate_100_Vb(name: str, with_lid: bool = False) -> Plate:
  """
This plate is a VWR PCR plate 96 well low-profile, half-skirted, ABI-FAST type plate.
VWR cat no. 89218-296
It is half-skirted so it must reside in another plate like a Cor_96_wellplate_360ul_Fb
  """
  
  return Plate(
    name=name,
    size_x=127.76,
    size_y=85.48,
    size_z=20.0,
    # lid=lid,
    model=VWR_96_wellplate_100_Vb.__name__,
    ordered_items=create_ordered_items_2d(
      Well,
      num_items_x=12,
      num_items_y=8,
      dx=10.25,  # keeping costar measurement
      dy=11.0,  # 7.77 keeping costar measurement
      dz=8.5, # how high is well above base
      item_dx=9.0,
      item_dy=9.0,
      size_x=5.75,  # measured
      size_y=5.4,  # measured
      size_z=16.3, # measured well depth, costar + VWR plate height
      material_z_thickness=0.5,
      bottom_type=WellBottomType.V,
      cross_section_type=CrossSectionType.CIRCLE,
      max_volume=100,
    ),
  )

In [ ]:
## ── build LH + deck ───────────────────────────────────────────────────────────
backend = STARBackend()
lh = LiquidHandler(backend=backend, deck=STARLetDeck())
await lh.setup(skip_autoload=True)          # faster while developing

# ── TIP CARRIER on rail 1 ──────────────────────────────────────────────────────
tip_car = TIP_CAR_480_A00("tip_car")
tip_car[0] = HTF(name="htf_300_tips")            # parked, not used
tip_car[1] = TIP_50ul_w_filter(name="tips_50f")  # filtered 50 µL tips
lh.deck.assign_child_resource(tip_car, rails=1)

# ── SOURCE carrier (deep-well plate) on rail 13 ───────────────────────────────
dwp_mod_src   = MFX_DWP_module_188042("dwp_mod_src")
flex_car_src  = MFX_CAR_L5_base("flex_car_src", modules={0: dwp_mod_src})
lh.deck.assign_child_resource(flex_car_src, rails=13)

# Deep-well plate pre-filled with 300 µL dye in wells A1–H1
src_plate = BioER_96_wellplate_Vb_2200ul("src_dye_plate")
dwp_mod_src.assign_child_resource(src_plate, location=0)

# ── TARGET carrier (qPCR plate) on rail 19 ────────────────────────────────────
dwp_mod_tgt   = MFX_DWP_module_188042("dwp_mod_tgt")
flex_car_tgt  = MFX_CAR_L5_base("flex_car_tgt", modules={0: dwp_mod_tgt})
lh.deck.assign_child_resource(flex_car_tgt, rails=19)

tgt_plate = VWR_96_wellplate_100_Vb("qpcr_plate")
dwp_mod_tgt.assign_child_resource(tgt_plate, location=0)
tgt_plate = lh.deck.get_resource("qpcr_plate")   # located object

In [ ]:
################################################################################
# Liquid-class parameters tuned for accuracy over speed
################################################################################
precise = dict(
    liquid_class="Water_Disp_Acc",     # Hamilton accurate aqueous LC
    aspirate_flow_rate=10,             # µL s⁻¹
    dispense_flow_rate=10,
    end_delay_seconds=0.5,
    blow_out=[0] *8
)

lld_surface_multi = dict(                       # pressure LLD
    lld_mode=[STARBackend.LLDMode.GAMMA] *8,
    lld_search_height=[30]*8,               # start 40 mm above deck
    immersion_depth=[1]*8,                  # go 1 mm below surface
    immersion_depth_direction=[0]*8
)

################################################################################
# Main protocol
################################################################################

# helper that always returns a Well, regardless of PLR build
def first_well(item):
    if isinstance(item, (list, tuple)):
        return item[0]          # unwrap the single-element list/tuple
    return item                 # already a Well

###############################################################################
# 0) pick up tips on every channel
###############################################################################
# tip_rack = lh.deck.get_resource("tips_50f")
# await lh.pick_up_tips(tip_rack["A3:H3"], use_channels=list(range(8)))

###############################################################################
# 1) build per-channel source wells (A1-H1) — flattened
###############################################################################
src_wells = [first_well(src_plate[f"{row}1"]) for row in "ABCDEFGH"]

###############################################################################
# 2) one-shot pre-wet, 30 µL on all eight channels
###############################################################################
await lh.aspirate(src_wells, vols=[30]*8, use_channels=list(range(8)),
                  **precise, liquid_height=[20]*8)
await lh.dispense(src_wells, vols=[30]*8, use_channels=list(range(8)),
                  **precise)

###############################################################################
# 3) iterate columns; simultaneous 8-channel transfers
###############################################################################
for col in range(1, 13):
    dst_wells = [first_well(tgt_plate[f"{row}{col}"]) for row in "ABCDEFGH"]

    await lh.aspirate(src_wells, vols=[20]*8, use_channels=list(range(8)),
                      **precise, **lld_surface_multi)
    await lh.dispense(dst_wells, vols=[20]*8, use_channels=list(range(8)),
                      **precise)

###############################################################################
# 4) blow-out & discard tips
###############################################################################
await lh.dispense(src_wells, vols=[0]*8, use_channels=list(range(8)),
                  blow_out=[1]*8)
await lh.discard_tips()



NameError: name 'get_star_liquid_class' is not defined

In [ ]:
# await lh.dispense(src, vols=[50], use_channels=list(range(8)), blow_out=[1])

In [ ]:
# await lh.dispense(src, vols=[50]*8, use_channels=[0], blow_out=[1])
# await lh.discard_tips()
# await lh.prepare_for_manual_channel_operation()
# await lh.stop()